Q: [1462. Course Schedule IV](https://leetcode.com/problems/course-schedule-iv/)

* **Objective**: Identify if for a given query request, `(pre, course)`, `pre` is a prerequisite for `course` or not.

* Approach 1 does that by checking for each possible pairing of courses, `(c1, c2)`, if `c1` can be a prerequisite of `c2` or not.
* Approach 2 seeks to build the complete set of prerequisites required for each course. The set is built while topologically sorting the courses.
* Approach 3 shows that topologically sorting them is unncessary-- as long as you're not interested in obtaining the complete set of prerequisites for each course.


**Approach 1: Floyd-Warshall Algorithm**

* **Key idea:** Use the [Floyd-Warshall Algorithm](https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm) to check for the *existence* of a path between *all possible node pairs*.	
* So that for each `(u, v)` in `queries`, you can directly ascertain whether `u` is a prerequisite of `v` or not.


* The prerequisite edges between all the node pairs `(i, j)` are stored in `pre`.
* `pre[i][j] = True` if (and only if the algorithm has determined that) there exists a path from `i` to `j`.

* The Algorithm:
	1. Check if there's a direct edge from `i` to `j`.
	2. If yes, then we can immediately set `pre[i][j] = True`.
	3. Else, check if there's a path from `i` to `j` through any course `k` (where `k` ranges from 0 to `numCourses`), and set the appropriate value to `pre[i][j]`.	
	4. Since we're computing `pre[i][j]`, the `i` and `j` for loops should be the inner ones. You can't simply interchange the order of the three loops.	 
		* Example: 
			* Say, `numCourses = 6`.
			* Say, you want to put `k` in the last loop.
			* To check for `pre[1][2]`, you would check for `pre[1][k] and pre[k][2] for k in range(6)`
			* Say, there's no direct prerequisite, `[1, 5]`, i.e., `[1, 5]` doesn't exist in `prerequisites`.
			* Then, we'd have
				* `pre[1][5] = False`, and, therefore,
				* `pre[1][2] = False`
			* And, `pre[1][2]` *would continue to remain* `False` *even if you later realized that there's a connection from course 1 to (say) course 3 and then to course 5.*

In [ ]:
class Solution:
    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:        
        pre = [[False] * numCourses for _ in range(numCourses)]
        
        for a, b in prerequisites:            
            pre[a][b] = True
                
        for k in range(numCourses):
            for i in range(numCourses):
                for j in range(numCourses):
                    if not pre[i][j]:
                        pre[i][j] = pre[i][k] and pre[k][j]
                        
        return [pre[u][v] for u, v in queries]

**Approach 2: Topological Sort**

* **A topological sort of a *directed acyclic graph* is an ordering of its vertices such that if there's an edge from vertex `u` to vertex `v` in the graph, then `u` apears before `v` in the sorted ordering.**
* Implemented below is an algorithm based on *breadth-first search*, known as [Kahn's algorithm](https://en.wikipedia.org/wiki/Topological_sorting#Kahn's_algorithm).

* Based on the given `prerequisites`, build a directed acyclic graph (represented by `graph` here).
	* It's a given constraint that `prerequisites` has no cycles.
* While building the `graph` also compute the **indegree** of each node.
	* **The indegree of a node is the number of edges incoming to it.**
	* Note that a "node" represents a "course" here.
	* So, a node with 0 indegree stands for a course with no prerequisites.
* For each `course`, `pre_reqs[course]` stores the set of prerequisites for that course.
* Store the nodes with 0 indegree in a queue, `q`.
* While `q` is not empty
	* Pop the front element. Since it's a node with 0 indegree, we name it `pre` (short for "prerequisite").
	* For each `course` that `pre` is a prerequisite for (Side note: observe this for loop to understand why this algorithm is based on BFS)
		* The set of prerequisites of `course` subsumes the set of prerequisites of `pre` (`|=` is the set union operator).
		* `in_degree[course] -= 1`.
		* If `in_degree[course] == 0`: enqueue `course` into `q`
* Finally, for `(pre, course)` in each query request, check if `pre` belongs to `pre_reqs[course]`.

In [ ]:
class Solution:
    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:
        graph = defaultdict(list)
        in_degree = [0] * numCourses
        pre_reqs = [set() for _ in range(numCourses)]
        
        for pre, course in prerequisites:
            graph[pre].append(course)
            in_degree[course] += 1
            pre_reqs[course].add(pre)
            
        q = deque([course for course, degree in enumerate(in_degree) if degree == 0])        
        while q:
            pre = q.popleft()
            for course in graph[pre]:
                pre_reqs[course] |= pre_reqs[pre]
                in_degree[course] -= 1
                if in_degree[course] == 0:
                    q.append(course)
            
        return [pre in pre_reqs[course] for pre, course in queries]

**Approach 3: Simply set union**

* For each `[pre, course]` in `prerequisites`
	* build the set of prerequisites required for `course`, and
	* the set of courses for which `pre` is a prequisite for.
* For a given course, `x`
	* `pre_course[x]` is the set of prerequisites for `x`.
	* `course_pre[x]` is the set of courses that `x` is a prerequisite for.
* Note:
	* We make only a single pass of `prerequisites`. 
	* So, these sets need not be complete.
* Given a query request, `(p, c)`, `return True if c in pre_course[p] or p in course_pre[c]`

In [ ]:
class Solution:
    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:                
        pre_course = defaultdict(set)
        course_pre = defaultdict(set)
        
        for p, c in prerequisites:
            pre_course[p].add(c)            
            pre_course[p] |= pre_course[c]
            course_pre[c].add(p)
            course_pre[c] |= course_pre[p]
                                
        return [c in pre_course[p] or p in course_pre[c] for p, c in queries]